# Template für Topic Modeling
Dieses Template soll dabei helfen, Topic Modeling automatisiert und einheitlich durchzuführen. 

## Allgemeine Vorbereitungsschritte

In [ ]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
# print(os.getcwd())
# os.chdir("C:/SV/HEX/Topic Modeling")

### Pakete laden

In [ ]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
import os
print(os.getcwd())
os.chdir("C:/Users/mhu/Documents/github/topic_model_it/")

import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer 
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

### Seed setzen
Wir setzen einen festen Seed, um Zufallszahlen in NumPy und PyTorch reproduzierbar zu machen, sowohl auf der CPU als auch auf der GPU (falls verfügbar). Das stellt sicher, dass Berechnungen mit zufälligen Operationen bei wiederholter Ausführung dieselben Ergebnisse liefern.

In [ ]:
seed = 40  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

### Datensätze einlesen
Der Trainings- und der Test-Datensatz werden hier eingelesen. Als Faustregel gilt, der Trainingsdatensatz sollte 80% und der Test-Datensatz 20% des Volumens ausmachen. 
Der Trainings-Datensatz wird für das trainieren / fitten des Modells verwendet. Der Test-Datensatz beinhaltet eine (in diesem Fall manuell erstellte) sogenannte "Ground Truth". Dies ist der Goldstandard, anhand dessen das Modell auf Performance hin überprüft wird. 

In [ ]:
# Training-Datensatz
training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

In [ ]:
# Test-Datensatz
test_data = pd.read_csv("data/bentley_labelled_data_edit.csv")
test_data = test_data.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
test_data['titel_kursbesch'] = test_data['veranstaltung_titel'] + ' ' + test_data['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
ground_truth = test_data["Label_ChatGPT"]  # Spalte im CSV mit manuell erstellten Begriffen, welche man als korrekt erachtet (i.d.R. einfach wichtige Wörter aus dem Text rauskopieren)

## NLP Vorbereitungsschritte
Zunächst werden die Trainingsdaten eingelesen und die gängigen Vorbereitungsschritte für NLP durchgeführt. Diese wären:
* Stopwords entfernen
* CountVectorizer spezifizieren

### Stopwords entfernen
Im Kontext des hier zu modellierenden Topic Modells werden sowohl standardisierte englische, deutsche als auch individuelle Stopwords generiert und im Objekt `sw` zusammengespielt.
Die Stopwords können je nach Anwendungsfall ergänzt oder reduziert werden.

In [11]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

### Lemmatisierung
Durch Lemmatisierung werden die Wörter in einheitliche Begriffe umgewandelt, sodass diese robuster werden. 

## Nur zum Nachlesen: Konfiguration, Training und Evaluation des Topic Models
Im Folgenden werden die einzelnen Schritte erläutert, wie man automatisiert ein, gemäss dem vordefinierten Goldstandard, möglichst performantes Modell erstellen kann. 
Dies dient lediglich als Nachschlagewerk, die Code-Zellen daher bitte __NICHT__ ausführen, während man das Topic Model automatisiert erstellen lassen will.
Die Anwendung für das aktuelle Topic Modeling wird im nächsten Block durchgeführt.


### Konfiguration des Modells

#### CountVectorizer
Der `CountVectorizer` ist ein wichtiges Werkzeug bei der Vorbereitung von Textdaten für ein Topic Model wie BERTopic. Ein Topic Model analysiert große Mengen an Text, um wiederkehrende Themen (Topics) zu erkennen. BERTopic kombiniert dabei Techniken aus der Natural Language Processing (NLP) mit Clustering-Methoden, um diese Themen zu extrahieren. Der `CountVectorizer` hilft dabei, den Text in eine numerische Darstellung umzuwandeln, die für das Modell nutzbar ist.

- `stop_words=sw`:  
  Stopwörter (z. B. "und", "der", "ein"), die keine inhaltliche Bedeutung tragen, werden entfernt. Dies stellt sicher, dass das Modell nur auf relevante Begriffe fokussiert ist und keine irrelevanten Wörter in die Themenbildung einfließen.

- `token_pattern=r'\b\w+\b'`:  
  Der reguläre Ausdruck sorgt dafür, dass nur ganze Wörter als Token betrachtet werden. Sonderzeichen oder isolierte Zahlen werden ausgeschlossen, da sie selten zur inhaltlichen Bedeutung beitragen.

- `ngram_range=(1, 3)`:  
  Es werden nicht nur einzelne Wörter (1-Gramme), sondern auch Wortkombinationen aus zwei oder drei aufeinanderfolgenden Wörtern (n-Gramme) berücksichtigt. Diese Phrasen wie "künstliche Intelligenz" oder "Datenanalyse" helfen BERTopic, kontextbezogene und aussagekräftige Themen zu identifizieren.

In [14]:
vectorizer = CountVectorizer(
    stop_words=irrelevant_terms,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
    token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
    ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

#### EmbeddingSettings

Die `EmbeddingSettings` definieren die Konfiguration für die Generierung von Text-Embeddings, die zur numerischen Darstellung von Textdaten verwendet werden. Diese Embeddings fassen semantische Ähnlichkeiten und Bedeutungen von Texten in einem Vektorraum zusammen und dienen als Grundlage für weitere Analysen, z. B. Clustering oder Themenmodellierung.

### Parameter der EmbeddingSettings

- `embedding_model`:  
  Gibt das Modell an, das zur Generierung der Embeddings verwendet wird. Hier wird die Klasse `SentenceTransformer` genutzt, die leistungsstarke vortrainierte Transformer-Modelle für die Textverarbeitung unterstützt.

- `model_name_or_path`:  
  Gibt den Pfad oder Namen des vortrainierten Modells an. In diesem Fall wird das Modell `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` verwendet, das speziell für mehrsprachige Anwendungen optimiert ist. Dieses Modell erzeugt kompakte und semantisch aussagekräftige Embeddings.


In [16]:
EmbeddingSettings = {
    "embedding_model": SentenceTransformer,
    "model_name_or_path": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
}

#### UMAPSettings

UMAP (Uniform Manifold Approximation and Projection) ist eine Technik zur Dimensionsreduktion, die häufig in der Verarbeitung von hochdimensionalen Daten, wie Text- oder Bilddaten, eingesetzt wird. Sie projiziert mehrdimensionale Daten in einen Raum mit geringerer Dimension, um Muster und Strukturen leichter zu erkennen. Die `UMAPSettings` definieren die Parameter, die das Verhalten und die Genauigkeit dieser Projektion steuern.


- `n_neighbors=15`:  
  Gibt die Anzahl der nächsten Nachbarn an, die für jeden Punkt berücksichtigt werden. Ein höherer Wert fokussiert auf größere Strukturen in den Daten, während ein niedrigerer Wert stärker lokale Muster betont.

- `n_components=5`:  
  Legt die Dimension des reduzierten Raumes fest. In diesem Fall werden die Daten in 5 Dimensionen projiziert, was hilft, wesentliche Eigenschaften der Daten zu erhalten.

- `min_dist=0.1`:  
  Bestimmt, wie nah Punkte im projizierten Raum beieinander liegen können. Ein niedriger Wert führt zu eng gepackten Clustern, während ein höherer Wert eine gleichmäßigere Verteilung ermöglicht.

- `metric="cosine"`:  
  Gibt die Distanzmetrik an, die verwendet wird, um die Ähnlichkeit zwischen Punkten zu berechnen. Der Kosinusabstand ist besonders geeignet für Textdaten oder hochdimensionale Vektoren.

- `random_state=13`:  
  Definiert einen Seed-Wert für den Zufallszahlengenerator, um reproduzierbare Ergebnisse sicherzustellen.


In [17]:
UMAPSettings = {
    "n_neighbors": 15,
    "n_components": 5,
    "min_dist": 0.1,
    "metric": "cosine",
    "random_state": 13
}

#### HDBSCANSettings

HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) ist ein Clustering-Algorithmus, der besonders gut mit komplexen Datensätzen umgehen kann. Er identifiziert Cluster auf der Basis von Dichte und ermöglicht es, Datenpunkte als *Rauschen* zu klassifizieren, wenn sie nicht zu einem Cluster gehören. Die `HDBSCANSettings` definieren die Parameter, die das Verhalten des Algorithmus steuern.

##### Parameter der HDBSCANSettings

- `min_samples=10`:  
  Gibt die minimale Anzahl von Datenpunkten an, die in der Nachbarschaft eines Punktes vorhanden sein müssen, damit er als Kernpunkt eines Clusters gilt. Ein höherer Wert macht den Algorithmus empfindlicher gegenüber Rauschen.

- `gen_min_span_tree=True`:  
  Erstellt einen minimalen `span tree`, der die hierarchische Struktur der Cluster visualisiert. Dies ist nützlich für die Analyse und Interpretation der Ergebnisse.

- `prediction_data=True`:  
  Ermöglicht die Generierung zusätzlicher Daten, die für die spätere Zuordnung neuer Punkte zu den Clustern verwendet werden können.

- `min_cluster_size=100`:  
  Gibt die minimale Größe eines Clusters an. Cluster mit weniger Datenpunkten werden als Rauschen betrachtet und nicht berücksichtigt.

In [18]:
HDBSCANSettings = {
    "min_samples": 3,
    "gen_min_span_tree": True,
    "prediction_data": True,
    "min_cluster_size": 10
}

#### BERTopicSettings

Die `BERTopicSettings` definieren die Konfiguration des BERTopic-Modells, das zur Themenmodellierung verwendet wird. BERTopic identifiziert wiederkehrende Themen in Textdaten und ermöglicht eine flexible Anpassung der Ergebnisse durch verschiedene Parameter.

#### Parameter der BERTopicSettings

- `top_n_words=10`:  
  Gibt an, wie viele Schlüsselwörter pro Thema angezeigt werden. Ein höherer Wert liefert detailliertere Informationen zu den Themen.

- `language="multilingual"`:  
  Setzt die Sprache für die Verarbeitung von Textdaten. Mit "multilingual" wird sichergestellt, dass Texte in mehreren Sprachen unterstützt werden.

- `n_gram_range=(1, 4)`:  
  Bestimmt den Bereich der n-Gramme (z. B. einzelne Wörter bis zu Vier-Wort-Kombinationen), die für die Themenanalyse berücksichtigt werden.

- `min_topic_size=100`:  
  Legt die minimale Anzahl von Dokumenten fest, die ein Thema enthalten muss, damit es berücksichtigt wird. Kleinere Mindestgrößen ermöglichen es, mehr spezialisierte Themen zu erkennen.

- `calculate_probabilities=True`:  
  Aktiviert die Berechnung von Wahrscheinlichkeiten, die die Zugehörigkeit von Dokumenten zu bestimmten Themen darstellen.

- `verbose=True`:  
  Aktiviert detaillierte Konsolenausgaben, die den Fortschritt des Modells anzeigen.

- `nr_topics=20`:  
  Setzt die Anzahl der finalen Themen auf 20. Diese Begrenzung wird durch die Reduktion ähnlicher Themen erreicht.


In [19]:
BERTopicSettings = {
    "top_n_words": 10,  # Mehr Top-Wörter, um relevantere Themen zu erfassen
    "language": "multilingual",
    "n_gram_range": (1, 3),  # Erweiterung des n-Gram-Bereichs
    "min_topic_size": 10,  # Kleinere Mindestgröße der Themen
    "calculate_probabilities": True,
    "verbose": True,
    "nr_topics": None,  # Festlegung der Anzahl der Themen
}

#### BERTopic: Initialisieren und trainieren

Im folgenden wird das Modell initialisiert.

##### Parameter BERTopic

- `ctfidf_model`:  
  Ein ClassTfidfTransformer wird mit dem Parameter `reduce_frequent_words=True` initialisiert. Dies reduziert den Einfluss hochfrequenter Wörter, die zusätzlich zu Stopwörtern das Modell verzerren könnten.

- `topic_model`:  
  Initialisierung des BERTopic-Modells mit verschiedenen benutzerdefinierten Einstellungen:
  - `ctfidf_model=ctfidf_model`:  
    Verwendet das zuvor definierte ClassTfidfTransformer-Modell, um hochfrequente Wörter zu behandeln.
  - `vectorizer_model=vectorizer`:  
    Übergibt den zuvor definierten CountVectorizer, der für die Tokenisierung und n-Gramm-Erstellung genutzt wird.
  - `embedding_model=EmbeddingSettings["embedding_model"](EmbeddingSettings["model_name_or_path"])`:  
    Erstellt ein Text-Embedding-Modell basierend auf den in EmbeddingSettings angegebenen Parametern.
  - `umap_model=UMAP(**UMAPSettings)`:  
    Nutzt UMAP zur Dimensionsreduktion mit den vorher definierten Einstellungen in UMAPSettings.
  - `hdbscan_model=HDBSCAN(**HDBSCANSettings)`:  
    Führt das Clustering mit HDBSCAN durch, basierend auf den Einstellungen in HDBSCANSettings.
  - `**BERTopicSettings`:  
    Übernimmt zusätzliche Parameter aus BERTopicSettings, wie die Anzahl der Themen oder die Sprache.

- Modelltraining und Transformation:  
  Der Datensatz wird durch das `fit_transform`-Verfahren verarbeitet:
  - `topics`: Enthält die identifizierten Themen für jedes Dokument.
  - `probs`: Liefert die Wahrscheinlichkeiten, mit denen ein Dokument zu einem Thema gehört.

- Themenanalyse:  
  `topic_model.get_topic_info()` gibt eine Übersicht der erkannten Themen, deren Häufigkeit und Repräsentation zurück. Hier werden die Top 50 Themen extrahiert und nach ihrer ID aufgelistet.


#### Definition Objective
Hier wird das Package "optuna" verwendet, durch welches die zu testenden Parameter festgelegt werden können und anhand einer definierten Metrik optimiert werden können.

## Anwendung: Konfiguration, Training und Evaluation des Topic Models
Hier muss alles innerhalb einer einzigen Code-Zelle erfolgen, da bei allen Konfigurationen variable Parameter vorkommen und wir diese durch das optuna-Package optimieren wollen.

In [ ]:
# Im Objective werden die verschiedenen Parameter-Settings gesetzt, über welche man optimieren möchte
def objective(trial):

  try:
    # Embedding Settings
    embedding_model_name = trial.suggest_categorical("embedding_model", ["paraphrase-multilingual-MiniLM-L12-v2", "paraphrase-mpnet-base-v2"])
    # UMAP Settings
    n_neighbors = trial.suggest_int("n_neighbors", 4, 14, 2)
    min_dist = trial.suggest_float("min_dist", 0.0, 0.2, step = 0.1)
    n_components = trial.suggest_int("n_components", 3, 8, 1)
    # HDBSCAN Settings
    min_cluster_size = trial.suggest_int("min_cluster_size", 3, 8, 1)
    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    # BERTopic Settings
    #nr_topics = trial.suggest_categorical("nr_topics", [5, 6, 7, 8, 9, 10, 15])
    diversity = trial.suggest_float("diversity", 0.0, 0.2, step = 0.1)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Konfiguration
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # CountVectorizer
    vectorizer = CountVectorizer(
      stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
      token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
      ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
    )

    # Embedding Settings  
    embedding_model = SentenceTransformer(embedding_model_name)
    
    # UMAP Settings
    umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="cosine", random_state=13)

    # HDBSCAN Settings
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, gen_min_span_tree=True, prediction_data=True)

    # Representation Settings
    representation_model = MaximalMarginalRelevance(diversity=diversity)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Training
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic initialisieren
    topic_model = BERTopic(
      embedding_model=embedding_model,
      #min_topic_size=10,
      #nr_topics=nr_topics, 
      language="multilingual",
      umap_model=umap_model,
      vectorizer_model=vectorizer,
      hdbscan_model=hdbscan_model,
      top_n_words = 30,
      representation_model=representation_model
    )

    # BERTopic trainieren
    topic_model_quanten = topic_model.fit(docs)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Evaluation
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic auf Test-Daten anwenden
    topics, probs = topic_model_quanten.transform(test_set)
    print(topic_model_quanten.get_topic_freq())

    # Outlier reduzieren
    topics = topic_model_quanten.reduce_outliers(test_set, topics)

    # Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
    dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
    dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
    dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

    # Goldstandard (Ground Truth) mit den Ergebnissen abgleichen und Score berechnen (Score = Anteil korrekter Topic-Zuweisungen)
    row_number = 0
    metric = 0
    while row_number < len(ground_truth):
      # Den Goldstandard in eine Liste von Keywords umwandeln
      ground_truth_current_iteration = ground_truth[row_number].split(", ")
      result_current_iteration = dataframe_with_results.at[row_number, "Representation"]

      # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
      if any(element in result_current_iteration for element in ground_truth_current_iteration):
              metric += 1
      else: metric += 0

      row_number = row_number+1

      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("---------------------------------------------------------------------------------------------------------------")

    metric_score = metric/row_number


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Ergebnis printen und Score returnen
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # Ergebnis printen
    #print("Modell evaluiert mit einem Score von ", metric_score, ". " \
    #"Verwendete Parameter: embedding model: ", embedding_model_name, ", nr_topics: ", nr_topics, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,
    #", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
  
    return metric_score 
  
  except Exception as e:
      print("Trial wird aufgrund eines Errors übersprungen")
      print("Verwendete Parameter: embedding model: ", embedding_model_name, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,\
      ", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
      print(e)
      raise optuna.TrialPruned()

In [ ]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)

print("Best parameters:", study.best_params)